**Table of contents**<a id='toc0_'></a>    
- [Import Statements](#toc1_1_)    
- [Importing the data](#toc1_2_)    
- [Dealing with Missing and Duplicated Data](#toc2_)    
  - [*Locating missing data*](#toc2_1_)    
  - [*Handling missing values*](#toc2_2_)    
  - [*Handling duplicate data*](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=5
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

**Read the official documentation on pandas DataFrames @ https://pandas.pydata.org/pandas-docs/stable/reference/frame.html**

**`Note:`** The notion of **chaining functions/methods** in pandas is similar to python.

DataFrames are **column oriented** unlike most common databases. And, **each column** in the dataframe is a **pandas series object**. So, any operation that can be performed on a pandas series object can be applied to a column too.

There are **two axes** for a dataframe commonly referred to as axis 0 and 1, or the **"index"** (or 'rows') axis and the **"columns"** axis respectively. Note that, when an **operation** is applied **along axis 0**, it is applied **down through all the rows for all the columns**. Likewise, operations **along axis 1** is applied **across the values in all the columns for all of the rows**.

### <a id='toc1_1_'></a>[Import Statements](#toc0_)

In [1]:
# import statements
import numpy as np
import pandas as pd

In [2]:
# view options
pd.set_option("display.max_columns", 14)
pd.set_option("display.max_rows", 8)

### <a id='toc1_2_'></a>[Importing the data](#toc0_)

- We will be exploring a dataset from a Siena College Poll in 2018. This data has rankings of United States Presidents in various attributes. These attributes are:

In [3]:
siena_2018_cols = """
• Bg = Background
• Im = Imagination
• Int = Integrity
• IQ = Intelligence
• L = Luck
• WR = Willing to take risks
• AC = Ability to compromise
• EAb = Executive ability
• LA = Leadership ability
• CAb = Communication ability
• OA = Overall ability
• PL = Party leadership
• RC = Relations with Congress
• CAp = Court appointments
• HE = Handling of economy
• EAp = Executive appointments
• DA = Domestic accomplishments
• FPA = Foreign policy accomplishments
• AM = Avoid crucial mistakes
• EV = Experts’ view
• O = Overall
"""

In [4]:
# reading from github url

# it is a good practice to define your index column when reading the data file.
# it is generally frowned upon if you don't have an index column

url = "https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv"
siena_2018 = pd.read_csv(url, index_col=0)

In [5]:
siena_2018.head(3)

,Seq.,President,Party,Bg,Im,Int,IQ,...,HE,EAp,DA,FPA,AM,EV,O
1,1,George Washington,Independent,7,7,1,10,...,1,1,2,2,1,2,1
2,2,John Adams,Federalist,3,13,4,4,...,13,15,19,13,16,10,14
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,...,20,4,6,9,7,5,5


In [6]:
# this will print all the column names, number of non null values in each column and the datatype of that column
siena_2018.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1 to 44
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Seq.       44 non-null     object
 1   President  44 non-null     object
 2   Party      44 non-null     object
 3   Bg         44 non-null     int64 
 4   Im         44 non-null     int64 
 5   Int        44 non-null     int64 
 6   IQ         44 non-null     int64 
 7   L          44 non-null     int64 
 8   WR         44 non-null     int64 
 9   AC         44 non-null     int64 
 10  EAb        44 non-null     int64 
 11  LA         44 non-null     int64 
 12  CAb        44 non-null     int64 
 13  OA         44 non-null     int64 
 14  PL         44 non-null     int64 
 15  RC         44 non-null     int64 
 16  CAp        44 non-null     int64 
 17  HE         44 non-null     int64 
 18  EAp        44 non-null     int64 
 19  DA         44 non-null     int64 
 20  FPA        44 non-null     int64 
 21  

In [7]:
# Datatype casting and renaming the columns
cols_list = [
    col.strip().split("=") for col in siena_2018_cols.strip().split(sep="•")[1:]
]

# we will replace the spaces in the full form with underscores (_)
siena_2018_cols_dict = {
    col_prev.strip(): col_full.strip().replace(" ", "_")
    for col_prev, col_full in cols_list
}
siena_2018 = siena_2018.rename(columns={"Seq.": "Seq"}).rename(
    columns=siena_2018_cols_dict
)
siena_2018 = siena_2018.astype({"Party": "category"})
siena_2018 = siena_2018.astype(
    {col_name: "uint8" for col_name in siena_2018.select_dtypes("int64").columns}
)
siena_2018 = siena_2018.assign(
    Average_rank=siena_2018.loc[:, "Background":"Experts’_view"]
    .sum(axis=1)
    .rank(method="dense")
    .astype("uint8"),
    Quartile_rank=lambda df_: pd.qcut(
        df_.Average_rank, 4, labels=["1st", "2nd", "3rd", "4th"]
    ),
)

--------------------------------------------

## <a id='toc2_'></a>[Dealing with Missing and Duplicated Data](#toc0_)

----------------------------------------------

### <a id='toc2_1_'></a>[*Locating missing data*](#toc0_)

- The `.isna()` method

Works similarly to series.isna() method. Returns a Boolean dataframe when used with dataframes.  

In [8]:
siena_2018.isna()

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,...,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts’_view,Overall,Average_rank,Quartile_rank
1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
42,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
43,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False
44,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False


We can use other methods such as **.any(), .all() etc.** in combination with the .isna() method to see whether there are any data at all missing from a column or whether all data in a column is Nan type etc.

In [9]:
siena_2018.isna().any()

Seq              False
President        False
Party            False
Background       False
                 ...  
Experts’_view    False
Overall          False
Average_rank     False
Quartile_rank    False
Length: 26, dtype: bool

To `count` how many data rows are `missing` from a particular column we can use the **df.isna().sum()**.

In [10]:
siena_2018.isna().sum()

Seq              0
President        0
Party            0
Background       0
                ..
Experts’_view    0
Overall          0
Average_rank     0
Quartile_rank    0
Length: 26, dtype: int64

To see what `percentage` of the data in a column is missing we can use something like, **df.isna().mean().mul(100)**.

In [11]:
siena_2018.isna().mean().mul(100)

Seq              0.0
President        0.0
Party            0.0
Background       0.0
                ... 
Experts’_view    0.0
Overall          0.0
Average_rank     0.0
Quartile_rank    0.0
Length: 26, dtype: float64

### <a id='toc2_2_'></a>[*Handling missing values*](#toc0_)

- The `.dropna(subset)` method 

We can use the good old .dropna() method to drop the rows with missing values. But note that, when using with dataframe, .dropna() will only drop the rows if it has Nan values in all the columns. To specify otherwise i.e, what columns to look at when dropping rows, **we can feed the subset parameter a list of column names (that we want it to look at for dropping Nan values)**.

- The `.fillna()` method

We can also use the .fillna() method to fill in the missing value. We can also define the filling method e.g, `ffill`, `bfill` etc. This method also takes a `value parameter` (value: scalar, dict, Series, or DataFrame) which will be used to fill the Nan values if specified. The **.mean(), .median(), .mode()** etc methods may come in handy when defining the value paramether.

- The `.interpolate()` method

This will replace Nan values with interpolation of the values around the missing value. This method comes in very handy when dealing with ordered data such as time series data.

- The `.where(cond, other)` method

Although not specific for handling missing values but this method is a powerful one for doing just that. This method **replaces values where the condition is False with corresponding value from 'other'**.

- The `.mask(cond, other)` method

Opposite of the .where() method in the sense that, this method will replace values **where the condition is True** with the corresponding value from 'other'. Equivalent to, **.where(~cond, other)**.

**`CAUTION:`** The data in each column of a dataframe usually represents different things. Thus applying methods such as .dropna(), .fillna(), .interpolate() is not logical and will bring no good (this is like, using a spoon for woodworking).

**So, the best approach is to treat each column differently as a separate series object, clean them, modify them and then adding/replacing them in a datafrmae using the .assign() method.**

### <a id='toc2_3_'></a>[*Handling duplicate data*](#toc0_)

- The `.duplicated(subset, keep)` method will return a boolean Series denoting duplicate rows

    - subset : column label or sequence of labels, optional. Only consider certain columns for identifying duplicates, by default uses all of the columns.

    - keep : {'first', 'last', False}. Determines which duplicates (if any) to mark.

        - first (default) : Mark duplicates as True except for the first occurrence.
        - last : Mark duplicates as True except for the last occurrence.
        - False : Mark all duplicates as True.


- The `.drop_duplicates(subset=None,keep='first', ignore_index=False)` method

If called without any parameters, it will drop only the rows that are complete copy of each other. The subset parameter lets us specify which columns to check when checking for duplicates.